<a href="https://colab.research.google.com/github/Sebabo7/backend-microservices/blob/main/microservice_api_gateway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask requests python-dotenv Flask-Limiter pyjwt

In [ ]:
%%writefile .env
SECRET_KEY=super_secret_key

In [ ]:
%%writefile app.py
from flask import Flask, jsonify, request
import requests
import jwt
import os
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__)

SECRET_KEY = os.getenv('SECRET_KEY', 'default_secret_key')

limiter = Limiter(get_remote_address, app=app, default_limits=["100 per minute"])

# route for login and JWT generation
@app.route('/login', methods=['POST'])
def login():
    user_data = request.json
    if user_data['username'] == 'test' and user_data['password'] == 'test':
        token = jwt.encode({'user': user_data['username']}, SECRET_KEY, algorithm='HS256')
        return jsonify({'token': token})
    return jsonify({'error': 'Unauthorized'}), 401

#  route for accessing services user, review or business
@app.route('/api/<service>/<id>', methods=['GET'])
@limiter.limit("50 per minute")
def gateway(service, id):
    token = request.headers.get('Authorization')
    try:
        jwt.decode(token, SECRET_KEY, algorithms=['HS256'])
    except jwt.ExpiredSignatureError:
        return jsonify({'error': 'Token has expired'}), 403
    except jwt.InvalidTokenError:
        return jsonify({'error': 'Invalid token'}), 403

    services = {
        'user': 'http://localhost:5001/user/',
        'review': 'http://localhost:5002/review/',
        'business': 'http://localhost:5003/business/'
    }

    if service in services:
        try:
            response = requests.get(services[service] + id, timeout=5)
            response.raise_for_status()
            return jsonify(response.json())
        except requests.exceptions.RequestException as e:
            return jsonify({'error': str(e)}), 503
    return jsonify({'error': 'Service not found'}), 404

if __name__ == "__main__":
    app.run(port=5000)


In [1]:
%%writefile README.md

# Microservice API Gateway

## Overview ##
This project demonstrates a microservice architecture using Flask, with rate limiting, JWT-based authentication, and an API gateway that communicates with multiple services.

## Features ##
- JWT-based authentication for secured access.
- Rate limiting to prevent abuse of API resources.
- API Gateway that routes to `user`, `review`, and `business` microservices.

## Setup ##

### Requirements ##
- Python 3.x
- Flask
- Requests
- Flask-Limiter
- PyJWT
- Python-dotenv

### Installation ##

1. Clone the repository:
   ```bash
   git clone https://github.com/Sebabo7/backend-microservices.git
   cd microservice-api-gateway

Writing README.md


In [2]:
%%writefile .gitignore
__pycache__/
*.py[cod]
*.egg-info/
venv/
.env

Writing .gitignore


In [3]:
%%writefile requirements.txt
Flask==2.1.1
requests==2.26.0
Flask-Limiter==1.3.1
PyJWT==2.3.0
python-dotenv==0.19.2

Writing requirements.txt
